# Exercice 8.5 - Jeu Mastermind

On remarquera que la problématique est symétrique : le résultat ne change pas si on intervertit secret et proposition.

## 8.5.1 Pseudo-code de `bienPaces()`

Il suffit de vérifier combien d'éléments de même indice sont égaux

```text
bienPlaces1()
------------ 
Secr: int[n] # Combinaison secrète
Test: int[n] # Tentative
------------
result <- 0
i <- 0
while i < n
    if Test[i] = Secr[i]
        result <- result + 1
    i <- i + 1
return result
```


In [30]:
from random import randint

def combinaison(n:int, b: int) -> list[int]:
    return [ randint(1,b) for _ in range(n)]

def bienPlaces(Secr: list[int], Test: list[int]):
    r = 0
    for i in range(len(Secr)):
        if Test[i]==Secr[i]:
            r += 1
    return r

## 8.5.2 Pseudo-code de `malPlaces()`

Si on définit par "mal placé" un élément de la proposition qui est présent dans le secret mais pas à la même place, le pseudo-code serait :

```text
malPlaces1()
------------
Secr: int[n]
Test: int[n]
------------
resultat <- 0

t <- 0
while t < n
    if Test[t] ≠ Secr[t] and Test[t] in Secr
        result <- result + 1
    t <- t + 1
return result
```

Il est évident que ce code est incorrect car les doubles dans la proposition ne sont pas gérés : on considère potentiellement plusieurs candidats mal placés pour un seul élément du secret (qui a éventuellement été résolu).

La problématique étant symétrique, on choisit de modifier la tentative en mettant à 0 les éléments ayant déjà fait l'objet d'une correspondance, mais les correspondances doivent inclure les bien placés car sinon on risque de valider un candidat mal placés alors qu'il existe un candidat bien placé ; il faut 
donc préalablement cribler les bien placés.

La fonction ci-dessous suppose donc que `Secr[i]≠Test[i]` car on a mis à 0 tous les éléments de `Test` bien placés.

```text
malPlaces2()
------------
Secr: int[n]
Test: int[n]
------------
resultat <- 0

s <- 0
while s < n
    t <- 0
    while t < n
        if Secr[s] = Test[t]
            resultat <- resultat + 1
            Test[t] = 0
    t <- t + 1
s <- s + 1
return resultat
```

## 8.5.3

Comme stipulé précédemment la fonction `malPlaces2()` présuppose que les éléments bien placés ont été précédement criblés, il faut donc modifier `bienPlaces()` pour ce faire.

La fonction `bienPlaces2()` effectue ce criblage et suppose que les modifications de `Test` (mise à 0 des éléments bien placés) persistent après l'exécution de la fonction.

```text
bienPlaces2()
------------ 
Secr: int[n] # Combinaison secrète
Test: int[n] # Tentative
------------
result <- 0
i <- 0
while i < n
    if Test[i] = Secr[i]
        result <- result + 1
        Test[i] = 0
    i <- i + 1
return result
```

On peut alors appeler successivement `bienPlaces2()` et `malPlaces2()` pour avoir le résultat

## 8.5.4

Comme `malPlaces2()` effectue 2 boucles impbriquées, on peut envisager de la modifier
pour calculer les deux valeurs _simultanément_.

```text
resultat2()
------------
Secr: int[n]
Test: int[n]
------------
bien <- 0
mal <- 0

s <- 0
while s < n
    if Test[s] = Secr[s]
        bien <- bien + 1
        Test[s] = 0
        next s
    else
        t <- 0    
        while t < n
            if Test[t]=Secr[s] and Test[t]≠Secr[t]
                Test[t] = 0
                mal <- mal + 1
                next s
            t <- t + 1
    s <- s + 1
return [bien,mal]
```

## 8.5.5 Implémentation de la méthode du TP

In [54]:
def resultat(S: list[int], T: list[int]) -> list[int]:
    bien = 0
    mal = 0

    for i in range(len(S)):
        if S[i] == T[i]:
            bien += 1
            T[i] = 0

    for s in S:
        for j in range(len(T)):
            if s == T[j]:
                mal += 1
                T[j] = 0
                break

    return [bien, mal]


In [55]:
tests = [
    [ [2,1,2,2], [1,3,3,3], [0,1] ],
    [ [1,1,2,2], [1,3,3,3], [1,0] ],
    [ [1,2,1,2], [3,1,3,3], [0,1] ],
    [ [1,2,2,2], [1,1,3,3], [1,0] ],
    [ [1,2,2,2], [3,1,1,3], [0,1] ],
    [ [1,1,2,2], [3,1,1,3], [1,1] ],
    [ [1,2,3,4], [1,2,3,4], [4,0] ],
    [ [1,2,3,4], [4,3,2,1], [0,4] ],

]

def execute_tests(func):
    for i in range(len(tests)):
        print(i+1,":", "+" if func(tests[i][0], tests[i][1]) == tests[i][2] else "KO")

execute_tests(resultat)

1 : +
2 : +
3 : +
4 : KO
5 : +
6 : +
7 : +
8 : +
